# Backpropagation: ejemplo sencillo


Vamos a calcular las derivadas de funciones en las que una operación está compuesta por otras. Por ejemplo, considera el modelo:

$$
\text{f}[x, \boldsymbol{\phi}] = \beta_3 + \omega_3 \cdot \cos\Bigl[\beta_2 + \omega_2 \cdot \exp\bigl[\beta_1 + \omega_1 \cdot \sin[\beta_0 + \omega_0 x]\bigr]\Bigr],
$$

donde los parámetros son $\boldsymbol{\phi} = \{\beta_0, \omega_0, \beta_1, \omega_1, \beta_2, \omega_2, \beta_3, \omega_3\}$.

Esta es una composición de las funciones $\cos[\cdot]$, $\exp[\cdot]$ y $\sin[\cdot]$.

Sus derivadas son:

$$
\frac{\partial \cos[z]}{\partial z} = -\sin[z], \quad \frac{\partial \exp[z]}{\partial z} = \exp[z], \quad \frac{\partial \sin[z]}{\partial z} = \cos[z].
$$

Supongamos ahora que tenemos una función de pérdida de mínimos cuadrados:

$$
\ell_i = (\text{f}[x_i, \boldsymbol{\phi}] - y_i)^2,
$$

y que conocemos los valores actuales de $\beta_0, \beta_1, \beta_2, \beta_3, \omega_0, \omega_1, \omega_2, \omega_3$, $x_i$ y $y_i$.

Podemos calcular $\ell_i$ directamente como composición de funciones, pues tenemos la entrada y los parámetros.

Ahora, queremos saber cómo cambia $\ell_i$ cuando hacemos un pequeño cambio en $\beta_0, \beta_1, \beta_2, \beta_3, \omega_0, \omega_1, \omega_2$ o $\omega_3$.

Es decir, queremos las derivadas:

$$
\frac{\partial \ell_i}{\partial \beta_0}, \quad \frac{\partial \ell_i}{\partial \beta_1}, \quad \frac{\partial \ell_i}{\partial \beta_2}, \quad \frac{\partial \ell_i}{\partial \beta_3}, \quad \frac{\partial \ell_i}{\partial \omega_0}, \quad \frac{\partial \ell_i}{\partial \omega_1}, \quad \frac{\partial \ell_i}{\partial \omega_2}, \quad \frac{\partial \ell_i}{\partial \omega_3}.
$$


In [ ]:
import numpy as np

Definimos la función y la pérdida para una etiqueta $y$:

In [ ]:
def fn(x, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3):
  return beta3+omega3 * np.cos(beta2 + omega2 * np.exp(beta1 + omega1 * np.sin(beta0 + omega0 * x)))

def loss(x, y, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3):
  diff = fn(x, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3) - y
  return diff**2

Ahora elegimos algunos valores para los parámetros, $y$, $x$, y calculamos el resultado de la función:

In [ ]:
beta0 = 1.0; beta1 = 2.0; beta2 = -3.0; beta3 = 0.4
omega0 = 0.1; omega1 = -0.4; omega2 = 2.0; omega3 = 3.0
x = 2.3; y = 2.0
l_i_func = loss(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3)
print('l_i=%3.3f'%l_i_func)

l_i=0.139


# Derivadas (a mano)

Podríamos calcular las expresiones para las derivadas a mano y escribir código para calcularlas directamente, pero algunas tienen expresiones muy complejas, incluso para esta ecuación original relativamente simple. Por ejemplo:
\begin{align}
\frac{\partial \ell_i}{\partial \omega_{0}} &=& -2 \left( \beta_3+\omega_3\cdot\cos\Bigl[\beta_2+\omega_2\cdot\exp\bigl[\beta_1+\omega_1\cdot\sin[\beta_0+\omega_0\cdot x_i]\bigr]\Bigr]-y_i\right)\nonumber \\
&&\hspace{0.5cm}\cdot \omega_1\omega_2\omega_3\cdot x_i\cdot\cos[\beta_0+\omega_0 \cdot x_i]\cdot\exp\Bigl[\beta_1 + \omega_1 \cdot \sin[\beta_0+\omega_0\cdot x_i]\Bigr]\nonumber\\
&& \hspace{1cm}\cdot \sin\biggl[\beta_2+\omega_2\cdot \exp\Bigl[\beta_1 + \omega_1 \cdot \sin[\beta_0+\omega_0\cdot x_i]\Bigr]\biggr].
\end{align}

In [ ]:
dldbeta3_func = 2 * (beta3 +omega3 * np.cos(beta2 + omega2 * np.exp(beta1+omega1 * np.sin(beta0+omega0 * x)))-y)
dldomega0_func = -2 *(beta3 +omega3 * np.cos(beta2 + omega2 * np.exp(beta1+omega1 * np.sin(beta0+omega0 * x)))-y) * \
              omega1 * omega2 * omega3 * x * np.cos(beta0 + omega0 * x) * np.exp(beta1 +omega1 * np.sin(beta0 + omega0 * x)) *\
              np.sin(beta2 + omega2 * np.exp(beta1+ omega1* np.sin(beta0+omega0 * x)))

Comprobemos si son correctas con diferencias finitas:

In [ ]:
dldomega0_fd = (loss(x,y,beta0,beta1,beta2,beta3,omega0+0.00001,omega1,omega2,omega3)-loss(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3))/0.00001

print('dydomega0: Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0_func,dldomega0_fd))

dydomega0: Function value = 5.246, Finite difference value = 5.246


El código para calcular $\partial \ell_i / \partial \omega_0$ es muy lioso y es fácil cometer errores. Además, algunas partes se repiten (por ejemplo, el término $\sin[\cdot]$), es decir, existe redundancia en los cálculos. El objetivo de este ejercicio práctico es calcular las derivadas de una manera mucho más sencilla. Habrá dos pasos:

**Paso 1:** Se escriben las ecuaciones como serie de cálculos intermedios:

\begin{align}
f_{0} &=& \beta_{0} + \omega_{0} x_i\nonumber\\
h_{1} &=& \sin[f_{0}]\nonumber\\
f_{1} &=& \beta_{1} + \omega_{1}h_{1}\nonumber\\
h_{2} &=& \exp[f_{1}]\nonumber\\
f_{2} &=& \beta_{2} + \omega_{2} h_{2}\nonumber\\
h_{3} &=& \cos[f_{2}]\nonumber\\
f_{3} &=& \beta_{3} + \omega_{3}h_{3}\nonumber\\
l_i &=& (f_3-y_i)^2
\end{align}

se calculan y almacenan dichos valores intermedios porque los necesitaremos. En resumen, hacemos un **forward pass**.

In [ ]:
############################## Completar ##############################
# Calcula los términos f_k y h_k

f0 = beta0 + (omega0 * x)
h1 = np.sin(f0)
f1 = beta1 + (omega1 * h1)
h2 = np.exp(f1)
f2 = beta2 + (omega2 * h2)
h3 = np.cos(f2)
f3 = beta3 + (omega3 * h3)
l_i = (f3-y)**2


In [ ]:
# Comprueba que son correctos:
print("f0: true value = %3.3f, your value = %3.3f"%(1.230, f0))
print("h1: true value = %3.3f, your value = %3.3f"%(0.942, h1))
print("f1: true value = %3.3f, your value = %3.3f"%(1.623, f1))
print("h2: true value = %3.3f, your value = %3.3f"%(5.068, h2))
print("f2: true value = %3.3f, your value = %3.3f"%(7.137, f2))
print("h3: true value = %3.3f, your value = %3.3f"%(0.657, h3))
print("f3: true value = %3.3f, your value = %3.3f"%(2.372, f3))
print("l_i original = %3.3f, l_i from forward pass = %3.3f"%(l_i_func, l_i))


f0: true value = 1.230, your value = 1.230
h1: true value = 0.942, your value = 0.942
f1: true value = 1.623, your value = 1.623
h2: true value = 5.068, your value = 5.068
f2: true value = 7.137, your value = 7.137
h3: true value = 0.657, your value = 0.657
f3: true value = 2.372, your value = 2.372
l_i original = 0.139, l_i from forward pass = 0.139


**Paso 2:** Se calculan las derivadas de $\ell_i$ con respecto a los valores intermedios, pero en orden inverso:

\begin{align}
\quad \frac{\partial \ell_i}{\partial f_3}, \quad \frac{\partial \ell_i}{\partial h_3}, \quad \frac{\partial \ell_i}{\partial f_2}, \quad
\frac{\partial \ell_i}{\partial h_2}, \quad \frac{\partial \ell_i}{\partial f_1}, \quad \frac{\partial \ell_i}{\partial h_1},  \quad\text{y} \quad \frac{\partial \ell_i}{\partial f_0}.
\end{align}

La primera es fácil:

\begin{equation}
\frac{\partial \ell_i}{\partial f_{3}} = 2 (f_3-y).
\end{equation}

La segunda se puede calcular aplicando la regla de la cadena:

\begin{equation}
\frac{\partial \ell_i}{\partial h_{3}} =\frac{\partial f_{3}}{\partial h_{3}} \frac{\partial \ell_i}{\partial f_{3}} .
\end{equation}

El lado izquierdo no dice cómo cambia $\ell_i$ cuando cambia $h_{3}$. El lado derecho nos dice que podemos descomponer esto en (i) cómo cambia $\ell_i$ cuando cambia $f_{3}$ y cómo cambia $f_{3}$ cuando cambia $h_{3}$. Así que se obtiene una cadena de eventos: $h_{3}$ cambia $f_{3}$, lo que cambia $\ell_i$, y las derivadas representan los efectos de esta cadena. Observa que ya calculamos la primera de estas derivadas, que es $2(f_3 - y)$. Calculamos $f_{3}$ en el paso 1. El segundo término es la derivada de $\beta_{3} + \omega_{3}h_{3}$ con respecto a $h_3$, que es simplemente $\omega_3$.

Podemos seguir de esta manera calculando la derivadas de las salidas con respecto a estas cantidades intermedias:
\begin{align}
\frac{\partial \ell_i}{\partial f_{2}} &=& \frac{\partial h_{3}}{\partial f_{2}}\left(
\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)
\nonumber \\
\frac{\partial \ell_i}{\partial h_{2}} &=& \frac{\partial f_{2}}{\partial h_{2}}\left(\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}}\right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{1}} &=& \frac{\partial h_{2}}{\partial f_{1}}\left( \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial h_{1}} &=& \frac{\partial f_{1}}{\partial h_{1}}\left(\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{0}} &=& \frac{\partial h_{1}}{\partial f_{0}}\left(\frac{\partial f_{1}}{\partial h_{1}}\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right).
\end{align}

En cada caso, ya hemos calculado todos los términos excepto el último en el paso anterior, y el último término es sencillo de evaluar. En resumen, hemos hecho un **backward pass**.

In [ ]:
############################## Completar ##############################
# Backward pass: Calcula las derivadas de la salida con respecto a las computaciones intermedias h_k y f_k

# Ya calculadas:
dldf3 = 2 * (f3 - y)  # \frac{\partial \ell_i}{\partial f_3}
dldh3 = omega3 * dldf3  # \frac{\partial \ell_i}{\partial h_3}

# Calcula las derivadas restantes:
dldf2 = -np.sin(f2) * (dldh3)
dldh2 = omega2 * dldf2

dldf1 = h2 * dldh2
dldh1 = omega1 * dldf1

dldf0= np.cos(f0)* dldh1

In [ ]:
# Comprueba que son correctos:
print("dldf3: true value = %3.3f, your value = %3.3f"%(0.745, dldf3))
print("dldh3: true value = %3.3f, your value = %3.3f"%(2.234, dldh3))
print("dldf2: true value = %3.3f, your value = %3.3f"%(-1.683, dldf2))
print("dldh2: true value = %3.3f, your value = %3.3f"%(-3.366, dldh2))
print("dldf1: true value = %3.3f, your value = %3.3f"%(-17.060, dldf1))
print("dldh1: true value = %3.3f, your value = %3.3f"%(6.824, dldh1))
print("dldf0: true value = %3.3f, your value = %3.3f"%(2.281, dldf0))

dldf3: true value = 0.745, your value = 0.745
dldh3: true value = 2.234, your value = 2.234
dldf2: true value = -1.683, your value = -1.683
dldh2: true value = -3.366, your value = -3.366
dldf1: true value = -17.060, your value = -17.060
dldh1: true value = 6.824, your value = 6.824
dldf0: true value = 2.281, your value = 2.281


In [ ]:
############################## Completar ##############################
# Derivadas finales con respecto a los parámetros

dldbeta3 = dldf3
dldomega3 =h3* dldf3

dldbeta2 = dldf2
dldomega2 =h2 * dldf2

dldbeta1 = dldf2
dldomega1 = h1* dldf1

dldbeta0 = dldf0
dldomega0 =x*dldf0


In [ ]:
# Comprueba que son correctos:
print('dldbeta3: Your value = %3.3f, True value = %3.3f'%(dldbeta3, 0.745))
print('dldomega3: Your value = %3.3f, True value = %3.3f'%(dldomega3, 0.489))
print('dldbeta2: Your value = %3.3f, True value = %3.3f'%(dldbeta2, -1.683))
print('dldomega2: Your value = %3.3f, True value = %3.3f'%(dldomega2, -8.530))
print('dldbeta1: Your value = %3.3f, True value = %3.3f'%(dldbeta1, -17.060))
print('dldomega1: Your value = %3.3f, True value = %3.3f'%(dldomega1, -16.079))
print('dldbeta0: Your value = %3.3f, True value = %3.3f'%(dldbeta0, 2.281))
print('dldomega0: Your value = %3.3f, Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0, dldomega0_func, dldomega0_fd))

dldbeta3: Your value = 0.745, True value = 0.745
dldomega3: Your value = 0.489, True value = 0.489
dldbeta2: Your value = -1.683, True value = -1.683
dldomega2: Your value = -8.530, True value = -8.530
dldbeta1: Your value = -1.683, True value = -17.060
dldomega1: Your value = -16.079, True value = -16.079
dldbeta0: Your value = 2.281, True value = 2.281
dldomega0: Your value = 5.246, Function value = 5.246, Finite difference value = 5.246


Usando este método, podemos calcular las derivadas con bastante facilidad sin necesidad de computar expresiones muy complicadas.
En la próxima práctica, aplicaremos este mismo método a una red neuronal profunda.